# Age Calculator (.py)

Finds the ages of a list of participants. This requires two sheets (Time tested sheet and DOB sheet). 

*Ensure to change the pathway of the sheets and adjust date format if needed*

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
def convert_to_datetime_str(date_val):
    if isinstance(date_val, datetime):
        return date_val.strftime('%B %d, %Y')
    else:
        return date_val


def convert_testdate_to_datetime_str(testdate_val):
    if isinstance(testdate_val, datetime):  
        return testdate_val.strftime('%Y-%m-%d')

    formats = ["%B %d, %Y", "%Y-%m-%d"]
    for fmt in formats:
        try:
            return datetime.strptime(testdate_val, fmt).strftime('%Y-%m-%d')
        except ValueError:
            pass
    print(f"Failed parsing testdate: {testdate_val}.")
    return testdate_val


def compute_age(birthday_str, testdate_str):
    try:
        birthdate_dt = datetime.strptime(birthday_str, "%B %d, %Y")
        testdate_dt = datetime.strptime(testdate_str, "%Y-%m-%d")
        age = testdate_dt.year - birthdate_dt.year - (
                    (testdate_dt.month, testdate_dt.day) < (birthdate_dt.month, birthdate_dt.day))
        return age
    except Exception as e:
        print(f"Failed parsing birthdate: {birthday_str}, testdate: {testdate_str}. Error: {e}")
        return "Unknown"

birthdays_df = pd.read_excel('C:\\Users\\samhs\\PycharmProjects\\pythonProject\\Dash\\Age.xlsx')

birthdays_df['Date'] = birthdays_df['Date'].apply(convert_to_datetime_str)

xls = pd.ExcelFile('C:\\Users\\samhs\\PycharmProjects\\pythonProject\\Dash\\TimeTested.xlsx')
test_dates_dfs = [pd.read_excel(xls, sheet_name) for sheet_name in xls.sheet_names]

with pd.ExcelWriter('C:\\Users\\samhs\\PycharmProjects\\pythonProject\\Dash\\TimeTested_processed.xlsx') as writer:
    for test_df in test_dates_dfs:
        ages = []
        for index, row in test_df.iterrows():
            participant_id = row['ID']
            test_date = convert_testdate_to_datetime_str(row['Date_Tested'])

            # Check if the participant's birthday is in the birthdays dataframe
            if participant_id in birthdays_df['ID'].values:
                birth_date = birthdays_df[birthdays_df['ID'] == participant_id]['Date'].iloc[0]
                age = compute_age(birth_date, test_date)
            else:
                age = "Unknown"

            ages.append(age)

        test_df['Age'] = ages


        test_df.to_excel(writer, sheet_name=f"Processed_{xls.sheet_names[test_dates_dfs.index(test_df)]}", index=False)

print("Age Calculated")
